In [4]:
import sys, os
sys.path.append("../../../../tools/CeSpGRN/src")
import numpy as np
import g_admm as cg
import kernel
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIR = "../../../../data/mESC/"

In [5]:
counts = np.load(DATA_DIR + "X.npy")
idx = range(0, counts.shape[1])
counts = counts[:, idx]
X_pca= np.load(DATA_DIR + "X_pca.npy")

k = 25
bandwidth = 5
lamda = 0.1

In [6]:
K, K_trun = kernel.calc_kernel_neigh(X_pca, k = 5, bandwidth = bandwidth, truncate = True, truncate_param = k)
empir_cov = cg.est_cov(X = counts, K_trun = K_trun, weighted_kt = True)

final number of nearest neighbor (make connected): 5


In [7]:
cespgrn = cg.G_admm_minibatch(X=counts[:, None, :], K=K, pre_cov=empir_cov, batchsize = 100)
thetas = cespgrn.train(max_iters=2500, n_intervals=100, lamb=lamda)

start running batch 0
n_iter: 100, duality gap: 9.8671e-08, primal residual: 3.5484e-07, dual residual: 1.461327e-06, loss1: 1.182537e+04, loss2: 1.317282e+04, loss3: 0.0000e+00
Batch loss: loss1: 11825.36523, loss2: 13172.81641, loss3: 0.00000
Finished running batch 0, running time: 55.79 sec
start running batch 1
n_iter: 100, duality gap: 9.1846e-08, primal residual: 3.4219e-07, dual residual: 1.542164e-06, loss1: 1.180861e+04, loss2: 1.319254e+04, loss3: 0.0000e+00
Batch loss: loss1: 11808.61035, loss2: 13192.53906, loss3: 0.00000
Finished running batch 1, running time: 42.36 sec
start running batch 2
n_iter: 100, duality gap: 1.2058e-07, primal residual: 3.3685e-07, dual residual: 1.461828e-06, loss1: 1.181657e+04, loss2: 1.315532e+04, loss3: 0.0000e+00
Batch loss: loss1: 11816.56641, loss2: 13155.31836, loss3: 0.00000
Finished running batch 2, running time: 45.60 sec
start running batch 3
n_iter: 100, duality gap: 1.0729e-07, primal residual: 3.5098e-07, dual residual: 1.450034e-0

In [9]:
np.save("G_cespgrn.npy", thetas.reshape(thetas.shape[0], -1))